In [1]:
import pandas as pd
import numpy as np

weather_df = pd.read_csv("/Users/neelesh/Documents/FS-Sem3/deep_learning/project-technical_paper/transformers_for_time_series_forecasting/nb/weather/weather.csv")
weather_df

,Unnamed: 0,Date Time,p (mbar),T (degC),rh (%),sh (g/kg),Tpot (K),Tdew (degC),VPmax (mbar),VPact (mbar),...,ST008 (degC),ST016 (degC),ST032 (degC),ST064 (degC),ST128 (degC),SM008 (%),SM016 (%),SM032 (%),SM064 (%),SM128 (%)
0,0,01.01.2020 00:10:04,1019.28,-0.02,96.7,3.61,271.66,-0.48,6.10,5.90,...,2.08,2.88,3.90,6.14,8.81,32.50,30.09,31.64,21.55,29.76
1,1,01.01.2020 00:20:04,1019.19,0.04,96.6,3.62,271.72,-0.43,6.13,5.92,...,2.07,2.88,3.90,6.14,8.80,32.50,30.09,31.64,21.55,29.76
2,2,01.01.2020 00:30:04,1019.11,0.10,97.2,3.66,271.78,-0.29,6.15,5.98,...,2.06,2.87,3.90,6.14,8.81,32.50,30.09,31.64,21.55,29.76
3,3,01.01.2020 00:40:04,1019.04,0.13,97.7,3.68,271.82,-0.19,6.16,6.02,...,2.06,2.87,3.90,6.13,8.81,32.50,30.09,31.64,21.55,29.76
4,4,01.01.2020 00:50:04,1018.98,0.00,97.9,3.66,271.69,-0.29,6.11,5.98,...,2.05,2.87,3.90,6.13,8.80,32.50,30.09,31.64,21.55,29.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52529,26328,31.12.2020 23:20:00,988.15,2.16,85.8,3.86,276.25,0.04,7.14,6.13,...,2.65,3.24,4.16,6.09,8.67,37.31,33.10,33.01,30.75,31.80
52530,26329,31.12.2020 23:30:00,988.13,2.15,88.6,3.99,276.24,0.47,7.13,6.32,...,2.66,3.24,4.16,6.09,8.67,37.32,33.10,33.01,30.75,31.80
52531,26330,31.12.2020 23:40:00,988.07,2.03,88.8,3.96,276.13,0.39,7.07,6.28,...,2.66,3.24,4.16,6.09,8.67,37.32,33.10,33.01,30.75,31.80
52532,26331,31.12.2020 23:50:00,988.08,1.93,88.5,3.92,276.03,0.24,7.02,6.22,...,2.67,3.25,4.16,6.09,8.67,37.32,33.09,33.01,30.76,31.80


In [2]:
#python -u run.py --data_path weather.csv --model FEDformer --pred_len 192 --loss adaptive

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Load your weather data
# (Assuming 'weather_df' is a DataFrame with your weather data)

# Define your look-back window and horizon
lookback_window = 96
horizon = 192

# Preprocess the data
def create_sequences(data, lookback, horizon):
    sequences, targets = [], []
    for i in range(len(data) - lookback - horizon + 1):
        sequence = data.iloc[i:i + lookback].values
        target = data.iloc[i + lookback:i + lookback + horizon]['T (degC)'].values
        sequences.append(sequence)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Create sequences and targets
sequences, targets = create_sequences(weather_df, lookback_window, horizon)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, targets, test_size=0.2, random_state=42)

# Convert the data to NumPy arrays and handle any missing or non-numeric values
X_train = sequences.astype(np.float32)
X_test = sequences.astype(np.float32)
y_train = targets.astype(np.float32)
y_test = targets.astype(np.float32)

# Create DataLoader for training and testing data
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

# Create DataLoader for training and testing data
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define your transformer model
class WeatherTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(WeatherTransformer, self).__init__()
        self.transformer = nn.Transformer(input_dim, num_heads, num_encoder_layers=num_layers, dim_feedforward=hidden_dim, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src):
        src = self.transformer(src)
        output = self.fc(src)
        return output

# Define hyperparameters
input_dim = 31  # Number of input features
output_dim = 192  # Output dimension for the horizon
hidden_dim = 256
num_layers = 2
num_heads = 4
dropout = 0.2
learning_rate = 0.001
epochs = 10

# Initialize and train your model
model = WeatherTransformer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate training loss for this epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {average_loss:.4f}")

# Evaluate the model
model.eval()
mse = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        predictions = model(inputs)
        mse += criterion(predictions, targets)

mse /= len(test_loader)
print(f"Mean Squared Error on Test Data: {mse:.4f}")